## Task 1. Run a batch data to get the loss value

ignore the bert mask

In [1]:
import torch
import torch.nn as nn
import json,time
import customize_data
import utils
import numpy as np
import esm.esmfold.v1.esmfold as ESM
import modules 

/opt/conda/envs/esmfold/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_file = "data/chain_set.jsonl"
dataset = customize_data.StructureDataset(data_file)
print(f"bad seq:{dataset.discard['bad_chars']}\ntoo long:{dataset.discard['too_long']}")
dataLoader = customize_data.StructureDataloader(dataset,2,num_workers=1)
batch_example = next(iter(dataLoader))

1cc1.L {'U'} SQAATPAADGKVKISIDPLTRVEGHLKIEVEVKDGKVVDAKCSGGMFRGFEQILRGRDPRDSSQIVQRICGVCPTAHCTASVMAQDDAFGVKVTTNGRITRNLIFGANYLQSHILHFYHLAALDYVKGPDVSPFVPRYANADLLTDRIKDGAKADATNTYGLNQYLKALEIRRICHEMVAMFGGRMPHVQGMVVGGATEIPTADKVAEYAARFKEVQKFVIEEYLPLIYTLGSVYTDLFETGIGWKNVIAFGVFPEDDDYKTFLLKPGVYIDGKDEEFDSKLVKEYVGHSFFDHSAPGGLHYSVGETNPNPDKPGAYSFVKAPRYKDKPCEVGPLARMWVQNPELSPVGQKLLKELYGIEAKKFRDLGDKAFSIMGRHVLRAEETWLTAVAVEKWLKQVQPGAETYVKSEIPDAAEGTGFTEAPRGALLHYLKIKDKKIENYQIVSATLWNANPRDDMGQRGPIEEALIGVPVPDIKNPVNVGRLVRSYDPULGCAVH
1gp1.A {'U'} AAALAAAAPRTVYAFSARPLAGGEPFNLSSLRGKVLLIENVASLUGTTVRDYTQMNDLQRRLGPRGLVVLGFPCNQFGHQENAKNEEILNCLKYVRPGGGFEPNFMLFEKCEVNGEKAHPLFAFLREVLPTPSDDATALMTDPKFITWSPVCRNDVSWNFEKFLVGPDGVPVRRYSRRFLTIDIEPDIETLLSQGASA
1nth.A {'O'} MTFRKSFDCYDFYDRAKVGEKCTQDDWDLMKIPMKAMELKQKYGLDFKGEFIPTDKDMMEKLFKAGFEMLLECGIYCTDTHRIVKYTEDEIWDAINNVQKEFVLGTGRDAVNVRKRSVGDKAKPIVQGGPTGSPISEDVFMPVHMSYALEKEVDTIVNGVMTSVRGKSPIPKSPYEVLAAKTETRLIKNACAMAGRPGMGVOGPETSLSAQGNISADCTGGMTCTDSHEVSQLNELKIDLDAISVIAHYKGNSDIIMDEQMPI

In [3]:
# Reading the data file and initialize the esm-inpainting class
model_path = "/root/.cache/torch/hub/checkpoints/esmfold_3B_v1.pt"
model_data = torch.load(str(model_path), map_location="cuda:0") #读取一个pickle文件为一个dict
cfg = model_data["cfg"]["model"]
model = modules.esm_inpaint(cfg) # make an instance
model_state = model_data["model"]
model.esmfold.load_state_dict(model_state, strict=False)

_IncompatibleKeys(missing_keys=['esm.embed_tokens.weight', 'esm.layers.0.self_attn.k_proj.weight', 'esm.layers.0.self_attn.k_proj.bias', 'esm.layers.0.self_attn.v_proj.weight', 'esm.layers.0.self_attn.v_proj.bias', 'esm.layers.0.self_attn.q_proj.weight', 'esm.layers.0.self_attn.q_proj.bias', 'esm.layers.0.self_attn.out_proj.weight', 'esm.layers.0.self_attn.out_proj.bias', 'esm.layers.0.self_attn.rot_emb.inv_freq', 'esm.layers.0.self_attn_layer_norm.weight', 'esm.layers.0.self_attn_layer_norm.bias', 'esm.layers.0.fc1.weight', 'esm.layers.0.fc1.bias', 'esm.layers.0.fc2.weight', 'esm.layers.0.fc2.bias', 'esm.layers.0.final_layer_norm.weight', 'esm.layers.0.final_layer_norm.bias', 'esm.layers.1.self_attn.k_proj.weight', 'esm.layers.1.self_attn.k_proj.bias', 'esm.layers.1.self_attn.v_proj.weight', 'esm.layers.1.self_attn.v_proj.bias', 'esm.layers.1.self_attn.q_proj.weight', 'esm.layers.1.self_attn.q_proj.bias', 'esm.layers.1.self_attn.out_proj.weight', 'esm.layers.1.self_attn.out_proj.bias'

In [4]:
model = model.to("cuda")
# 11735MiB / 32510MiB

In [5]:
#         dis_embed: torch.Tensor,
#         bb_frame: torch.Tensor,
#         aa: torch.Tensor,
#         mask: T.Optional[torch.Tensor] = None,
#         residx: T.Optional[torch.Tensor] = None,
#         masking_pattern: T.Optional[torch.Tensor] = None,
#         num_recycles: T.Optional[int] = None,

In [6]:
utils.recur_print(batch_example)

{'coord': 'torch.Size([2, 379, 4, 3])_torch.float32',
 'seq': 'torch.Size([2, 379])_torch.int64',
 'bert_mask_fraction': 'torch.Size([2, 1])_torch.float32',
 'bert_mask': 'torch.Size([2, 379])_torch.float32',
 'padding_mask': 'torch.Size([2, 379])_torch.float32'}

In [7]:
with torch.no_grad():
    for key in batch_example.keys():
        batch_example[key] = batch_example[key].to("cuda")
    output = model(coord=batch_example['coord'],mask=(batch_example['padding_mask']).to(torch.float32),S=batch_example['seq'])

torch.float32
coord cuda:0
 mask cuda:0
 mask_frame cuda:0
embedding cuda:0
 bb_frame cuda:0
torch.Size([2, 379, 379, 128])
Recycling:0-----------
Folding Trunk
Structure Module
ipa module 0
ipa dropout
ipa layernormalization
transition
update backbone frames
All side chain frames
xyz prediction
ipa module 1
ipa dropout
ipa layernormalization
transition
update backbone frames
All side chain frames
xyz prediction
ipa module 2


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemmStridedBatched( handle, opa, opb, m, n, k, &alpha, a, lda, stridea, b, ldb, strideb, &beta, c, ldc, stridec, num_batches)`

In [ ]:

batch_example['padding_mask'].shape

torch.Size([1, 190])

In [10]:
torch.__version__

'1.13.0+cu117'

In [11]:
torch.version.cuda

'11.7'